# 自动求导
假设我们想对函数 $y=2x^Tx$ 关于列向量x求导

In [4]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在计算y关于x的梯度之前，需要一个地方来存储梯度

In [5]:
x.requires_grad_(True)  # 等价于 x=torch.arange(4.0, requires_grad=True)
x.grad  # 访问梯度 默认值是None

现在计算y

In [6]:
y = 2*torch.dot(x, x)  # 点积是相同位置的按元素乘积的和
y

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数 自动计算y关于x每个分量的梯度

In [7]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [8]:
x.grad == 4*x

tensor([True, True, True, True])

In [11]:
# 在默认情况下，pytorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

深度学习中 目的不是计算微分矩阵 而且批量中每个样本单独计算的偏导数之和 一般都是对标量求导

In [14]:
# 对于非标量用 backward 需要传入一个 gradient 参数，该参数指定
x.grad.zero_()
y = x*x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()  # 先得到 y=x1**2+x2**2+x3**2+x4**2, 然后再分别求导?
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图以外

In [15]:
x.grad.zero_()
y = x*x
u = y.detach() #让u不再是x的函数 视为常数
z = u*x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [16]:
x.grad.zero_()
y.sum().backward()
x.grad == 2*x

tensor([True, True, True, True])

即使构建函数的计算图需要通过Python控制流（例如，条件，循环或任意函数调用)，我们仍然可以计算得到的变量的梯度

In [20]:
def f(a):
    b = a*2
    while b.norm() < 1000:  # norm 范数
        b = b*2
    if b.sum() > 0:
        c = b
    else:
        c = 100*b
    return c


a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d/a

tensor(True)

a= torch.randn(size=(),requires_grad=True)
d = f(a)           
# 看完函数f(a)    得d=f(a)=c,    c为b   或者  b*100
# 且 b 为a*2  或者   a*(n个2      2n=实数)
# 综上得  d  最终结果都是   此处用A代表实数   a*A  
# 则   d=f(a)=Aa
# 对 d  求导得 A
d.backward()
a.grad == d/a
# d=Aa  故d/a = 实数A     a的导数   a.grad  =  实数A